In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Fact Extraction and VERification task

In [2]:
import os
import numpy as np
import csv
import pandas as pd
import json
import string
import collections
import itertools
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
!pip install jsonlines
import jsonlines as jl
#!pip install wikipedia
#import wikipedia

In [3]:
import torch
# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [0]:
#If the file is downloaded to the format .json1 you have to rename the file and remove the "1" at the end.
#Otherwise it's impossible to read the file
datafile_fever = '/content/drive/My Drive/Colab Notebooks/data/train.json'
datafile_shared_task = '/content/drive/My Drive/Colab Notebooks/data/shared_task_dev.json'
wiki_pages_folder = '/content/drive/My Drive/Colab Notebooks/data/wiki-pages/wiki-pages'

In [5]:
df_train = pd.read_json(datafile_fever, lines=True)
df_train.shape

(145449, 5)

In [10]:
df_task = pd.read_json(datafile_shared_task, lines=True)
df_task.shape

(19998, 5)

Importation of wiki pages data

In [0]:
from os.path import isfile, join
files = [f for f in os.listdir(wiki_pages_folder) if isfile(join(wiki_pages_folder, f))]

In [7]:
with jl.open(wiki_pages_folder+"/"+files[10]) as f:
  for line in tqdm(f.iter()):
      print(line)

Output hidden; open in https://colab.research.google.com to view.

In [8]:
with jl.open(wiki_pages_folder+"/"+files[10]) as f:
  for line in tqdm(f.iter()):
    print("\n",line)
    for key in line:
      print(key)
    break

0it [00:00, ?it/s]


 {'id': 'Anna_Szabolcsi', 'text': 'Anna Szabolcsi is a linguist whose research has focused on semantics , syntax , and the syntax-semantics interface . She was born and educated in Hungary . She has been a research fellow at the Research Institute for Linguistics of the Hungarian Academy of Sciences , Budapest , professor at UCLA , and is currently professor at New York University .   Szabolcsi was one of the first to propose the determiner phrase hypothesis and alongside Mark Steedman and others initiated research in combinatory categorial grammar . More recently she has worked on quantification , islands , polarity , verbal complexes , and overt nominative subjects in infinitival complements . ', 'lines': '0\tAnna Szabolcsi is a linguist whose research has focused on semantics , syntax , and the syntax-semantics interface .\tsemantics\tsemantics\tsyntax\tsyntax\n1\tShe was born and educated in Hungary .\n2\tShe has been a research fellow at the Research Institute for Linguistics of 

## Data exploration

In [9]:
df_train['evidence'][5]

[[[151831, 166598, 'Homeland_-LRB-TV_series-RRB-', 0],
  [151831, 166598, 'Prisoners_of_War_-LRB-TV_series-RRB-', 0]]]

In [11]:
#df_task.head()
print(df_train['claim'][0])
print(df_train['evidence'][0][0][0][3])

Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.
7


In [12]:
df_train[5:10].head()

,id,verifiable,label,claim,evidence
5,129629,VERIFIABLE,SUPPORTS,Homeland is an American television spy thrille...,"[[[151831, 166598, Homeland_-LRB-TV_series-RRB..."
6,149579,NOT VERIFIABLE,NOT ENOUGH INFO,Beautiful reached number two on the Billboard ...,"[[[173384, None, None, None]]]"
7,229289,NOT VERIFIABLE,NOT ENOUGH INFO,Neal Schon was named in 1954.,"[[[273626, None, None, None]]]"
8,33078,VERIFIABLE,SUPPORTS,The Boston Celtics play their home games at TD...,"[[[49158, 58489, Boston_Celtics, 3]], [[49159,..."
9,6744,VERIFIABLE,SUPPORTS,The Ten Commandments is an epic film.,"[[[23513, 28977, The_Ten_Commandments_-LRB-195..."


In [16]:
df_train['evidence'] = df_train['evidence'].replace("-LRB-","(")
df_train['evidence'] = df_train['evidence'].replace("-RRB-",")")
df_train[0:10].head()

,id,verifiable,label,claim,evidence
0,75397,VERIFIABLE,SUPPORTS,Nikolaj Coster-Waldau worked with the Fox Broa...,"[[[92206, 104971, Nikolaj_Coster-Waldau, 7], [..."
1,150448,VERIFIABLE,SUPPORTS,Roman Atwood is a content creator.,"[[[174271, 187498, Roman_Atwood, 1]], [[174271..."
2,214861,VERIFIABLE,SUPPORTS,"History of art includes architecture, dance, s...","[[[255136, 254645, History_of_art, 2]]]"
3,156709,VERIFIABLE,REFUTES,Adrienne Bailon is an accountant.,"[[[180804, 193183, Adrienne_Bailon, 0]]]"
4,83235,NOT VERIFIABLE,NOT ENOUGH INFO,System of a Down briefly disbanded in limbo.,"[[[100277, None, None, None]]]"


Looking into the wikipedia jsonl

In [13]:
texts = []


with jl.open(wiki_pages_folder+"/"+files[70]) as f:
  for line in tqdm(f.iter()):
    texts.append(line)
    #for i in range(len(line['id'])):
    #  if line['id'][i]== 'Nikolaj_Coster-Waldau':
    #    texts.append(line)
    #    print(i)

50000it [00:00, 54301.10it/s]


In [30]:
texts = pd.DataFrame(texts)
texts.head()

,id,text,lines
0,"New_Caledonian_independence_referendum,_2018",An independence referendum will be held in New...,0\tAn independence referendum will be held in ...
1,Negaativa,Negaativa is a trio of female Brazilian Portug...,0\tNegaativa is a trio of female Brazilian Por...
2,"Nesøya,_Akershus",Nesøya is a small island in the municipality o...,0\tNesøya is a small island in the municipalit...
3,Newton_College_-LRB-Peru-RRB-,Newton College is a Peruvian British co-educat...,0\tNewton College is a Peruvian British co-edu...
4,National_Junior_Angus_Show,The National Junior Angus Show is the premier ...,0\tThe National Junior Angus Show is the premi...


In [15]:
#texts["text"][0]
print(texts["lines"][0])

0	An independence referendum will be held in New Caledonia , no later than in November 2018 .	independence referendum	independence referendum	New Caledonia	New Caledonia
1	Voters will be given the choice of remaining part of France or becoming an independent country .	France	France
2	


In [31]:
for i in range(len(texts)):
  if(texts['id'][i]== 'Nikolaj_Coster-Waldau'):
    print(i)

25816


In [0]:
der = texts['text'][25816].split(".")

In [19]:
der

['Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰʌsd̥ɐ ˈʋald̥ɑʊ̯ -RSB- ; born 27 July 1970 -RRB- is a Danish actor , producer and screenwriter ',
 ' He graduated from Danish National School of Theatre in Copenhagen in 1993 ',
 " Coster-Waldau 's breakthrough performance in Denmark was his role in the film Nightwatch -LRB- 1994 -RRB- ",
 ' Since then he has appeared in numerous films in his native Scandinavia and Europe in general , including Headhunters -LRB- 2011 -RRB- and A Thousand Times Good Night -LRB- 2013 -RRB- ',
 '   In the United States , his debut film role was in the war film Black Hawk Down -LRB- 2001 -RRB- , playing Medal of Honor recipient Gary Gordon ',
 ' He then played Detective John Amsterdam in the short-lived Fox television series New Amsterdam -LRB- 2008 -RRB- , as well as appearing as Frank Pike in the 2009 Fox television film Virtuality , originally intended as a pilot ',
 ' He became widely known to a broad audience for his current role as Ser Jaime Lannister , 

# Find evidence

In [0]:
#path, dirs, files = next(os.walk(wiki_pages_folder))
file_count = len(files)

def findEvidence(evidence, verifiable):
  found = False
  for i in tqdm(range(len(files))):
    with jl.open(wiki_pages_folder+"/"+files[i]) as f:
      for j, article in enumerate(f.iter()):   
        # wiki[283015][title_id] = wiki['id'][title_id].split("\n")
        # wiki[283015][title_id] = wiki['id'][title_id].replace(" ", "_")
        if article['id']==evidence:
          print(article['id'])
          print("file : ", files[i]," and article number : ", j)
          found = True
          break
    if found:
      break

def getEvidence(evidence, verifiable)  :
  dataset = []
  for i in tqdm(range(len(evidence))):
    if verifiable[i]=="VERIFIABLE":
      for j in range(len(evidence[i])):
        sentence = ""
        for k in range(len(evidence[i][j])):
          title = evidence[i][j][k][2]
          line = evidence[i][j][k][3]
          article = ""
          found = False
          for x in range(len(files)):
            with jl.open(wiki_pages_folder+"/"+files[x]) as f:
              for y, art in enumerate(f.iter()): 
                if(title == art["id"]):
                  article = art["text"]
                  found = True
                  break
              if(found == True):
                break
          article = article.split(".")
          print(article)
          sentence += ' ' + article[line]
        dataset.append([evidence[i][j][k][0],evidence[i][j][k][1],sentence])
  return dataset

In [21]:
df_train['evidence'][0:5][0]

[[[92206, 104971, 'Nikolaj_Coster-Waldau', 7],
  [92206, 104971, 'Fox_Broadcasting_Company', 0]]]

In [22]:
df_train['evidence'][0:110]

0      [[[92206, 104971, Nikolaj_Coster-Waldau, 7], [...
1      [[[174271, 187498, Roman_Atwood, 1]], [[174271...
2                [[[255136, 254645, History_of_art, 2]]]
3               [[[180804, 193183, Adrienne_Bailon, 0]]]
4                         [[[100277, None, None, None]]]
                             ...                        
105    [[[136980, 151810, 21_Jump_Street_-LRB-film-RR...
106    [[[247949, 248240, Philadelphia_Museum_of_Art,...
107    [[[158017, 172644, Dustin_Hoffman, 6], [158017...
108     [[[260555, 258945, Deadpool_-LRB-film-RRB-, 1]]]
109                       [[[247476, None, None, None]]]
Name: evidence, Length: 110, dtype: object

In [23]:
wiki = []
with jl.open(wiki_pages_folder+'/wiki-001.jsonl') as f:
  for line in f.iter():
      wiki.append(line)
print("wiki shape : (", len(wiki), ",", len(wiki[0]), ")")

wiki shape : ( 50000 , 3 )


In [24]:
# wiki['id'][283015] = wiki['id'][283015].split("\n")
# wiki['id'][283015] = wiki['id'][283015].replace(" ", "_")
print(wiki[28015]["id"])

1992_Pacific_typhoon_season


In [25]:
findEvidence('1992_Pacific_typhoon_season',0)

  0%|          | 0/109 [00:00<?, ?it/s]

1992_Pacific_typhoon_season
file :  wiki-001.jsonl  and article number :  28015


In [28]:
test = df_train['evidence'][0:110][0][0][0][2]
print(test)

Nikolaj_Coster-Waldau


In [0]:
data = getEvidence(df_train['evidence'][0:110],df_train['verifiable'][0:110])


  0%|          | 0/110 [00:00<?, ?it/s]

['Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰʌsd̥ɐ ˈʋald̥ɑʊ̯ -RSB- ; born 27 July 1970 -RRB- is a Danish actor , producer and screenwriter ', ' He graduated from Danish National School of Theatre in Copenhagen in 1993 ', " Coster-Waldau 's breakthrough performance in Denmark was his role in the film Nightwatch -LRB- 1994 -RRB- ", ' Since then he has appeared in numerous films in his native Scandinavia and Europe in general , including Headhunters -LRB- 2011 -RRB- and A Thousand Times Good Night -LRB- 2013 -RRB- ', '   In the United States , his debut film role was in the war film Black Hawk Down -LRB- 2001 -RRB- , playing Medal of Honor recipient Gary Gordon ', ' He then played Detective John Amsterdam in the short-lived Fox television series New Amsterdam -LRB- 2008 -RRB- , as well as appearing as Frank Pike in the 2009 Fox television film Virtuality , originally intended as a pilot ', ' He became widely known to a broad audience for his current role as Ser Jaime Lannister , in the


  1%|          | 1/110 [01:30<2:44:46, 90.70s/it]

['The Fox Broadcasting Company -LRB- often shortened to Fox and stylized as FOX -RRB- is an American English language commercial broadcast television network that is owned by the Fox Entertainment Group subsidiary of 21st Century Fox ', ' The network is headquartered at the 20th Century Fox studio lot on Pico Boulevard in the Century City section of Los Angeles , with additional major offices and production facilities at the Fox Television Center in nearby West Los Angeles and the Fox Broadcasting Center in the Yorkville neighborhood of Manhattan , New York City ', ' It is the third largest major television network in the world based on total revenues , assets , and international coverage ', '   Launched on October 9 , 1986 as a competitor to the Big Three television networks -LRB- ABC , NBC and CBS -RRB- , Fox went on to become the most successful attempt at a fourth television network ', ' It was the highest-rated broadcast network in the 18 -- 49 demographic from 2004 to 2012 , and 

In [0]:
print(data)

In [0]:
#import re
def train_findEvidence(evidence,verifiable):
#trouver la ou les phrases
#si plusieurs, les concatener
#ensuite mettre l'evidence dans un data_frame en gardant les id 
#si le titre contient des parentheses on a l'a dans le cul faut supprimer

  dataset = []
  for i in tqdm(range(len(evidence))):
    if verifiable[i]=="VERIFIABLE":
      for j in range(len(evidence[i])):
        sentence = ""
        for k in range(len(evidence[i][j])):
          url = evidence[i][j][k][2]
          if "-LRB-" not in url:
            line = evidence[i][j][k][3]
            #print("url : ",type(url))
            url = url.replace("-","_")
            print(url)
            #suggest = wikipedia.suggest(url)
            #if suggest !=None:
            try:
              article = (wikipedia.page(url).content).split(".")
              break
            except PageError:
              sentence += ' '+article[line]
          dataset.append([evidence[i][j][k][0],evidence[i][j][k][1],sentence])
  return dataset

In [0]:
data = train_findEvidence(df_train['evidence'][0:110],df_train['verifiable'][0:110])
print(data)






  0%|          | 0/110 [00:00<?, ?it/s]

Nikolaj_Coster_Waldau







  1%|          | 1/110 [00:00<01:23,  1.30it/s]

Roman_Atwood
Roman_Atwood







  2%|▏         | 2/110 [00:02<01:46,  1.01it/s]

...






 89%|████████▉ | 98/110 [01:30<00:08,  1.41it/s]

Khmer_Empire







 92%|█████████▏| 101/110 [01:31<00:05,  1.74it/s]

Peter_Davison


NameError: ignored

In [0]:
df_train['evidence'][100:110]

100                  [[[66965, 77763, Khmer_Empire, 0]]]
101    [[[199623, 209302, On_the_Road_-LRB-film-RRB-,...
102                       [[[133857, None, None, None]]]
103    [[[106250, 119689, Paramore_-LRB-album-RRB-, 2]]]
104    [[[171118, 184635, Peter_Davison, 1]], [[17111...
105    [[[136980, 151810, 21_Jump_Street_-LRB-film-RR...
106    [[[247949, 248240, Philadelphia_Museum_of_Art,...
107    [[[158017, 172644, Dustin_Hoffman, 6], [158017...
108     [[[260555, 258945, Deadpool_-LRB-film-RRB-, 1]]]
109                       [[[247476, None, None, None]]]
Name: evidence, dtype: object

In [0]:
print(data[5])

[180804, 193183, ' \n\n\n== Discography ==\n\n\n=== Studio albums ===\n\n\n=== Guest appearances ===\n\n\n=== Unreleased/shelved ===\nUnapologetic (2010)\n\n\n== Filmography ==\n\n\n=== Film roles ===\n\n\n=== Television roles ===\n\n\n=== Web ===\n\n\n=== Music video appearances ===\n\n\n== Awards and nominations ==\nNote: The year given is the year of the ceremony\n\n\n== References ==\n\n\n== External links ==\n\n{}}\nAdrienne Bailon on IMDb']


In [0]:
import wikipediaapi as wikipedia

In [0]:
wiki = wikipedia.Wikipedia('en')
# page_py = wiki_wiki.page('On_the_Road_(film)')
# print("Page - Title: %s" % page_py.title)
#     # Page - Title: Python (programming language)

# print("Page - Summary: %s" % page_py.summary)
    # Page - Summary: Python is a widely used high-level programming language for

In [0]:
tg = wiki.page('Nikolaj_Coster-Waldau').text.split(".")

In [59]:
print(tg)

['Nikolaj Coster-Waldau (Danish pronunciation: [ˈne̝koˌlɑjˀ ˈkʰʌstɐ ˈvæltɑw]; born 27 July 1970) is a Danish actor, producer and screenwriter', ' He graduated from the Danish National School of Performing Arts in Copenhagen in 1993', " Coster-Waldau's breakthrough performance in Denmark was his role in the film Nightwatch (1994)", ' Since then he has appeared in numerous films in his native Scandinavia and Europe in general, including Headhunters (2011) and A Thousand Times Good Night (2013)', '\nIn the U', 'S, his debut film role was in the war film Black Hawk Down (2001), playing Medal of Honor recipient Gary Gordon', ' He then played Detective John Amsterdam in the short-lived Fox television series New Amsterdam (2008), as well as appearing as Frank Pike in the 2009 Fox television film Virtuality, originally intended as a pilot', ' He became widely known for his role as Jaime Lannister in the HBO fantasy series Game of Thrones, for which he received Primetime Emmy Award nominations 

In [0]:
#import re
def train_findEvidence(evidence,verifiable):
#trouver la ou les phrases
#si plusieurs, les concatener
#ensuite mettre l'evidence dans un data_frame en gardant les id 
#si le titre contient des parentheses on a l'a dans le cul faut supprimer

  dataset = []
  for i in tqdm(range(len(evidence))):
    if verifiable[i]=="VERIFIABLE":
      for j in range(len(evidence[i])):
        sentence = ""
        for k in range(len(evidence[i][j])):
          url = evidence[i][j][k][2]
          #if "-LRB-" not in url:
          line = evidence[i][j][k][3]
          #print("url : ",type(url))
          url = url.replace("-LRB-","(")
          url = url.replace("-RRB-",")")
          url = url.replace("-","_")
          print(url)
          #suggest = wikipedia.suggest(url)
          #if suggest !=None:
          article = wiki.page(url).summary.split(".")
          sentence += ' '+article[line]
        dataset.append([evidence[i][j][k][0],evidence[i][j][k][1],sentence])
  return dataset

In [0]:
data = train_findEvidence(df_train['evidence'][0:110],df_train['verifiable'][0:110])
print(data)








  0%|          | 0/110 [00:00<?, ?it/s]

Nikolaj_Coster_Waldau









  1%|          | 1/110 [00:00<00:47,  2.29it/s]

Fox_Broadcasting_Company
Roman_Atwood









  2%|▏         | 2/110 [00:00<00:39,  2.70it/s]

...

Boston_Celtics
The_Ten_Commandments_(1956_film)









  9%|▉         | 10/110 [00:01<00:20,  4.79it/s]

The_Ten_Commandments_(1956_film)
Tetris


  9%|▉         | 10/110 [00:01<00:19,  5.04it/s]


IndexError: ignored

In [0]:
tg = wiki.page('Nikolaj_Coster-Waldau').text.split(".")


In [0]:
tg[0:10]

['Nikolaj Coster-Waldau (Danish pronunciation: [ˈne̝koˌlɑjˀ ˈkʰʌstɐ ˈvæltɑw]; born 27 July 1970) is a Danish actor, producer and screenwriter',
 ' He graduated from the Danish National School of Performing Arts in Copenhagen in 1993',
 " Coster-Waldau's breakthrough performance in Denmark was his role in the film Nightwatch (1994)",
 ' Since then he has appeared in numerous films in his native Scandinavia and Europe in general, including Headhunters (2011) and A Thousand Times Good Night (2013)',
 '\nIn the U',
 'S, his debut film role was in the war film Black Hawk Down (2001), playing Medal of Honor recipient Gary Gordon',
 ' He then played Detective John Amsterdam in the short-lived Fox television series New Amsterdam (2008), as well as appearing as Frank Pike in the 2009 Fox television film Virtuality, originally intended as a pilot',
 ' He became widely known for his role as Jaime Lannister in the HBO fantasy series Game of Thrones, for which he received Primetime Emmy Award nomin

In [0]:
df_train['evidence'][10]

[[[269479, 265800, 'Tetris', 18]]]